# Urine data illustrative example processing

# Package loading and file path specification

In [1]:
import specxplore.importing
import matchms
import matchms.filtering
import ms2query
import os
import pandas as pd
import numpy as np

In [2]:
# Relative input and output paths
# Input paths
models_and_library_folder_path = os.path.join("models", "pos")
input_mgf_filepath = os.path.join("data", "data_urine_output", "urine_plus_phophe.mgf")

# ms2query required paths
input_data_folder  = os.path.join("data", "data_urine_output")
mgf_filename       = os.path.join("urine_plus_phophe.mgf")
output_ms2query_directory = os.path.join("data", "data_urine_output") # folder to which ms2query puts the results csv
output_ms2query_filepath = os.path.join("data", "data_urine_output", "urine_plus_phophe.csv") # ms2query csv file name derived from input spectrum filename

# metadata for standards
metadata_csv_filepath =  os.path.join("data", "data_phophe_output", "metadata_phophe_standards_pos_processed.csv")

# Output paths
output_filepath = os.path.join("data", "data_urine_output", "specxplore_urine.pickle")

# Loading spectral data

In [3]:
spectra_matchms = list(matchms.importing.load_from_mgf(input_mgf_filepath))

# Basic spectral data processing

In [4]:
spectra_matchms = specxplore.importing.apply_basic_matchms_filters_to_spectra(spectra_matchms)
# Additional noise removal filtering
spectra_matchms = [matchms.filtering.select_by_relative_intensity(spec, intensity_from = 0.01, intensity_to = 1) for spec in spectra_matchms]
spectra_matchms = [spec for spec in spectra_matchms if spec is not None and spec.peaks.mz.size >= 5] 
print("Number of spectra after additional filtering:", len(spectra_matchms))

Number of spectra prior to filtering:  4381
Number of spectra after to filtering:  4346
Number of spectra after additional filtering: 3818


In [5]:
# Check for sucess
for spec in spectra_matchms:
    assert all(spec.peaks.mz < 1000), "Peak with mz of 1000 or more found."
print('Assertions on excess peaks passed.')
# Check for sucess
for spec in spectra_matchms:
    assert spec.peaks.mz.size > 0, "Spectrum with insufficient number of peaks found."
print('Assertions on minimum peaks passed.')

Assertions on excess peaks passed.
Assertions on minimum peaks passed.


In [6]:
# Check for uniqueness of feature_ids 
feature_ids = [spec.get("feature_id") for spec in spectra_matchms]
assert len(feature_ids) == len(set(feature_ids))

# Loading metadata

In [7]:
standards_metadata = pd.read_csv(metadata_csv_filepath)
standards_metadata['feature_id'] = standards_metadata['feature_id'].astype('string')

# Run ms2query

To run or rerun ms2query, set the boolean to True. This step is deactivated as a default as it takes the longest.

In [8]:
if False:
        ms2library = ms2query.create_library_object_from_one_dir(models_and_library_folder_path)
        ms2query.run_ms2query_single_file(
                ms2library = ms2library, 
                folder_with_spectra = input_data_folder,
                spectrum_file_name = mgf_filename, 
                results_folder = output_ms2query_directory,
                settings = ms2query.utils.SettingsRunMS2Query())

# Align ms2query with feature_ids & extract analog classifications


In [9]:
raw_mgf_spectra = list(matchms.importing.load_from_mgf(input_mgf_filepath))
raw_data_spectrum_number = [iloc for iloc in range(1, len(raw_mgf_spectra)+1)]
raw_data_feature_ids = [spec.get('feature_id') for spec in raw_mgf_spectra]
raw_iloc_to_feature_id_mapping = pd.DataFrame({"feature_id": raw_data_feature_ids, "query_spectrum_nr" : raw_data_spectrum_number})
# run stuff for ms2query here or at separate file, you need the csv file here 
ms2query_annotation_table = pd.read_csv(output_ms2query_filepath)
ms2query_annotation_table = ms2query_annotation_table.merge(raw_iloc_to_feature_id_mapping, how = "left", on="query_spectrum_nr")
# renaming ms2query feature identifier column
ms2query_annotation_table["feature_id"] = ms2query_annotation_table["feature_id"].astype("string") # recasting to string type if not already

In [10]:
# extracting ms2query analog classification table for heuristic highlighting
ms2query_analog_classification = ms2query_annotation_table.loc[:, ['cf_superclass', 'cf_class', 'cf_subclass',
       'cf_direct_parent', 'npc_class_results', 'npc_superclass_results',
       'npc_pathway_results', 'feature_id']]

# Create basic specXplore session object

In [11]:
cosine_scores = specxplore.importing.compute_similarities_cosine(
    spectrum_list=spectra_matchms
)

In [12]:
ms2ds_scores = specxplore.importing.compute_similarities_ms2ds(
    spectrum_list=spectra_matchms, 
    model_path=models_and_library_folder_path
)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



Calculating vectors of reference spectrums: 100%|██████████| 3818/3818 [01:33<00:00, 40.74it/s]


In [13]:
s2v_scores = specxplore.importing.compute_similarities_s2v(
    spectrum_list=spectra_matchms, 
    model_path=models_and_library_folder_path
)

In [14]:
heuristic_score = np.maximum(cosine_scores, ms2ds_scores) # not used

In [15]:
# use the specXplore constructor to get a barebones session data object (not suitable for specXplore yet)
specxplore_urine = specxplore.importing.SessionData(
    spectra_matchms,
    models_and_library_folder_path,
    primary_score = ms2ds_scores,
    secondary_score = cosine_scores,
    tertiary_score = s2v_scores,
    score_names = ["ms2ds", "cosine", "spec2vec"]
)

# Run k-medoid clustering and t-SNE

In [16]:
# run kmedoid and tsne grid computations for assessing 
specxplore_urine.attach_kmedoid_grid(k_values=[8, 16, 24, 32, 64, 128])
specxplore_urine.attach_run_tsne_grid( perplexity_values=[5, 10, 20, 30, 40, 50, 60])

iloc Number-of-Clusters Silhouette-Score
0 8 0.201
1 16 0.191
2 24 0.207
3 32 0.195
4 64 0.219
5 128 0.211
iloc Perplexity Pearson-score Spearman-score
0 5 0.443 0.42
1 10 0.515 0.498
2 20 0.565 0.55
3 30 0.585 0.572
4 40 0.601 0.587
5 50 0.608 0.593
6 60 0.609 0.593


# Select k-medoid and t-SNE values

In [17]:
# select a particular iloc of the tsne grid with good distance preservation
specxplore_urine.select_tsne_coordinates(6) 
# select particular iloc(s) for kmedoid cluster assignments to add to class table
specxplore_urine.select_kmedoid_cluster_assignments([0,1,2,4]) 

# Attach ms2query classifications to class table

In [18]:
specxplore_urine.attach_addon_data_to_class_table(ms2query_analog_classification)

# Attach standard feature ids to highlight table

In [19]:
specxplore_urine.construct_highlight_table(standards_metadata["feature_id"].to_list()) 

# Attach metadata

In [20]:
specxplore_urine.attach_addon_data_to_metadata(standards_metadata)
specxplore_urine.attach_addon_data_to_metadata(ms2query_annotation_table)
specxplore_urine.metadata_table

,feature_id,spectrum_iloc,Unnamed: 0,compound_db_identity,compound_name,compound_annotation_score,mol_formula,adduct,precursor_mz,mz_diff_ppm,...,retention_index,smiles,cf_kingdom,cf_superclass,cf_class,cf_subclass,cf_direct_parent,npc_class_results,npc_superclass_results,npc_pathway_results
0,urine_1,0,not available,not available,not available,not available,not available,not available,not available,not available,...,not available,CCCN(CCOC1=C(Cl)C=C(Cl)C=C1Cl)C(=O)NC=O,Organic compounds,Benzenoids,Phenol ethers,not available,Phenol ethers,not available,not available,not available
1,urine_2,1,not available,not available,not available,not available,not available,not available,not available,not available,...,not available,not available,not available,not available,not available,not available,not available,not available,not available,not available
2,urine_5,2,not available,not available,not available,not available,not available,not available,not available,not available,...,not available,not available,not available,not available,not available,not available,not available,not available,not available,not available
3,urine_6,3,not available,not available,not available,not available,not available,not available,not available,not available,...,not available,CC[C@H](C)[C@@H](C(=O)N[C@@H](CC(C)C)C(=O)N[C@...,not available,not available,not available,not available,not available,Dipeptides; Tripeptides,Small peptides,Amino acids and Peptides
4,urine_16,4,not available,not available,not available,not available,not available,not available,not available,not available,...,not available,CC(C)C[C@@H](C(=O)O)NC(=O)[C@H](CO)N,Organic compounds,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",Dipeptides,Dipeptides,Small peptides,Amino acids and Peptides
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,14411,3813,72.0,not available,not available,not available,not available,not available,not available,not available,...,not available,not available,not available,not available,not available,not available,not available,not available,not available,not available
3814,14542,3814,74.0,S-equol: [M+H]+: 0.715,S-equol,0.71,C15H14O3,[M+H]+,243.10157,-0.5,...,not available,C1[C@H](COC2=C1C=CC(=C2)O)C3=CC=C(C=C3)O,Organic compounds,Phenylpropanoids and polyketides,Isoflavonoids,Isoflavans,Isoflavanols,Isoflavanones,Isoflavonoids,Shikimates and Phenylpropanoids
3815,14758,3815,75.0,Kaempferol: [M+H]+: 0.576,Kaempferol,0.58,C15H10O6,[M+H]+,287.05501,-1.15,...,not available,not available,not available,not available,not available,not available,not available,not available,not available,not available
3816,15017,3816,77.0,Hesperetin: [M+H]+: 0.691,Hesperetin,0.69,C16H14O6,[M+H]+,303.08631,-1.23,...,not available,O=C1C2=C(O)C=C(O)C=C2OC(C3=CC=C(OC)C(O)=C3)C1,Organic compounds,Phenylpropanoids and polyketides,Flavonoids,O-methylated flavonoids,4'-O-methylated flavonoids,Flavanones,Flavonoids,Shikimates and Phenylpropanoids


# Initialize and save specxplore running session

In [21]:
specxplore_urine.initialize_specxplore_session()
specxplore_urine.check_and_save_to_file(output_filepath)